<a href="https://colab.research.google.com/github/Berlingrad/MLinCyberSecurityProject/blob/XuDevelopment/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DNN Back Door Detector

## Preprocessing

###Data Download
Clone the CSAW competition repo and download provided datasets from the Google drive. Downloaded data are moved <code>/content/CSAW-HackML-2020/data</code> directory.

In [1]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

!gdown https://drive.google.com/uc?id=1-ay0w_hgTMo6ZNxb7CZhqxWZhkMj1I6Z
!gdown https://drive.google.com/uc?id=1A1CnWy2N1gjzWO6ZEUgiQ-LOAMgKvElL
!gdown https://drive.google.com/uc?id=1ZH7jMGEKhtmI-K0Jw-oKv843jxs-_kkI
!gdown https://drive.google.com/uc?id=1Sw77QLiBn-FuXpPXnas-Gdo3M_aQBD_H

!mv /content/*.h5 /content/CSAW-HackML-2020/data


Cloning into 'CSAW-HackML-2020'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 164 (delta 11), reused 18 (delta 6), pack-reused 141
Receiving objects: 100% (164/164), 79.56 MiB | 37.05 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Downloading...
From: https://drive.google.com/uc?id=1-ay0w_hgTMo6ZNxb7CZhqxWZhkMj1I6Z
To: /content/clean_test_data.h5
398MB [00:02, 148MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1A1CnWy2N1gjzWO6ZEUgiQ-LOAMgKvElL
To: /content/clean_validation_data.h5
716MB [00:03, 199MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZH7jMGEKhtmI-K0Jw-oKv843jxs-_kkI
To: /content/sunglasses_poisoned_data.h5
398MB [00:06, 59.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Sw77QLiBn-FuXpPXnas-Gdo3M_aQBD_H
To: /content/anonymous_1_poisoned_data.h5
637MB [00:16, 39.2MB/s]


In [2]:
import os
import gdown
import numpy as np
import random
import h5py
import glob2
import tempfile
import copy

import keras
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
!pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

##check
if not tf.config.experimental.list_physical_devices('GPU'):
    raise Exception("Please change runtime type to GPU")

     |████████████████████████████████| 174kB 5.2MB/s 


Functions for loading and decompresssing data and models from h5 format. If you added a dataset, simply run load_dataset with the path to the h5

In [3]:
def load_dataset(data_filename, keys=None):
    dataset = {}
    nClass = 1283
    with h5py.File(data_filename, 'r') as hf:
        if keys is None:
            for name in hf:
                dataset[name] = np.array(hf.get(name))
        else:
            for name in keys:
                dataset[name] = np.array(hf.get(name))
        dataset['data'] /= 255

        y = np.zeros((dataset['label'].shape[0], nClass))

        for i in range(dataset['label'].shape[0]):
            y[i][int(dataset['label'][i])] = 1

        dataset['data'] = dataset['data'].transpose((0,2,3,1))
        dataset['label'] = y
        x = dataset['data']

    return x, y, dataset

def load_models(model_dir):
    models = []
    names = {}

    for i, net in enumerate(glob2.glob("/content/CSAW-HackML-2020/models/*net.h5")):
        print("loading", net)
        weights = glob2.glob(net.split("_")[0]+"*weights.h5")[0]
       
        m = load_model(net)
        m.load_weights(weights)

        models.append(m)
        
        names[i] = os.path.basename(net).split(".")[0]
    return models, names

###Load Data

In [4]:
SG_poisoned_path = "/content/CSAW-HackML-2020/data/sunglasses_poisoned_data.h5"
xSG, ySG, SG_poisoned = load_dataset(SG_poisoned_path)

clean_validation_path = "/content/CSAW-HackML-2020/data/clean_validation_data.h5"
xValid, yValid, valid_clean = load_dataset(clean_validation_path)

clean_test_path = "/content/CSAW-HackML-2020/data/clean_test_data.h5"
xTest, yTest, test_clean = load_dataset(clean_test_path)

anon_poisoned_path = "/content/CSAW-HackML-2020/data/anonymous_1_poisoned_data.h5"
xAnon, yAnon, anon_poisoned = load_dataset(anon_poisoned_path)


####Mixed final test data
Mix the sunglass poinsoned images with clean test images from test set to create a final validation set 

In [5]:
##TODO
def mix_dataset(A, B):
    nSample = A['data'].shape[0] + B['data'].shape[0] 
    offset = A['data'].shape[0]
    xShape = (nSample, 55, 47, 3)
    yShape = (nSample, 1283+1)
    xOut = np.zeros(xShape)
    yOut = np.zeros(yShape)

    iOut = [i for i in range(nSample)]
    random.shuffle(iOut)

    for i in iOut:
        if i >= offset:

            xOut[i] = B['data'][i-offset]

            if np.argmax(B['label'][i-offset]) != 0:
                yOut[i] = np.append(B['label'][i-offset], [0])

            else:
                yOut[i][-1] = 0
            
        else:
            xOut[i] = A['data'][i]
            if np.argmax(A['label'][i]) != 0:
                yOut[i] = np.append(A['label'][i], [0])
            else:
                yOut[i][-1] = 0

    return xOut, yOut 

In [6]:
xSGmixed, ySGmixed = mix_dataset(SG_poisoned, test_clean)

####Load your own data

In [7]:
##Path_to_your_data = 
##yourX, yourY, yourDataset = load_dataset(Path_to_your_data)

###Load Models
Load all Badnet models under <code>/content/CSAW-HackML-2020/models</code> directory. 

In [8]:

model_dir = "/content/CSAW-HackML-2020/models"

models, model_names = load_models(model_dir)

model_names.values()


loading /content/CSAW-HackML-2020/models/anonymous_1_bd_net.h5
loading /content/CSAW-HackML-2020/models/multi_trigger_multi_target_bd_net.h5
loading /content/CSAW-HackML-2020/models/sunglasses_bd_net.h5
loading /content/CSAW-HackML-2020/models/anonymous_2_bd_net.h5


dict_values(['anonymous_1_bd_net', 'multi_trigger_multi_target_bd_net', 'sunglasses_bd_net', 'anonymous_2_bd_net'])

### Model Summary

In [9]:
##TODO
for i, model in enumerate(models):
    print("model ", i, ":")
    model.summary()

     

model  0 :
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
_________________________________________________________________________________

In [10]:
def validate_model(y, y_hat):
    ##compute model accuracy based vectorized output
    n = 0
    for i, v in enumerate(y):
        correct = 1 if np.argmax(v) == np.argmax(y_hat[i]) else 0
        n+= correct 
    return n/(i + 1)
    
    

####Evaluate loaded models on poisoned data.

 Note poisoned data will not involved in the repairing process. We are evaluating it to set baselines, Later we will use the results to validate our method 

In [11]:
for i, m in enumerate(models):
    predClean = m.predict(xTest)
    predSG = m.predict(xSG)
    predAnon = m.predict(xAnon)
    print("badnet", model_names[i], "accuracy on clean test set: ", validate_model(predClean, yTest))
    print("badnet", model_names[i], "accuracy on sunglass poisoned set: ", validate_model(predSG, ySG))
    print("badnet", model_names[i], "accuracy on anonumous poisoned set: ", validate_model(predAnon, yAnon))

badnet anonymous_1_bd_net accuracy on clean test set:  0.971862821512081
badnet anonymous_1_bd_net accuracy on sunglass poisoned set:  0.0
badnet anonymous_1_bd_net accuracy on anonumous poisoned set:  0.913971161340608
badnet multi_trigger_multi_target_bd_net accuracy on clean test set:  0.9600935307872175
badnet multi_trigger_multi_target_bd_net accuracy on sunglass poisoned set:  0.0
badnet multi_trigger_multi_target_bd_net accuracy on anonumous poisoned set:  0.0
badnet sunglasses_bd_net accuracy on clean test set:  0.9777864380358535
badnet sunglasses_bd_net accuracy on sunglass poisoned set:  0.9999220576773188
badnet sunglasses_bd_net accuracy on anonumous poisoned set:  0.0010717069368667187
badnet anonymous_2_bd_net accuracy on clean test set:  0.971862821512081
badnet anonymous_2_bd_net accuracy on sunglass poisoned set:  0.0
badnet anonymous_2_bd_net accuracy on anonumous poisoned set:  0.913971161340608


###Save pre-repaired results
For later poisoned image detection.

In [12]:
preds_preprocessed = [] ##test result of the badnets before repair

for i, m in enumerate(models):
    if "sunglasses" in model_names[i].lower(): ##only save output from sunglasses backdoored model to save RAM 
        pred = m.predict(xSGmixed)
        preds_preprocessed.append(pred)

    ##print(model_names[i], "accuracy on the clean testset before repairing: ", validate_model(yTest, pred))
'''
for m in models:
    pred = m.predict(xSG)
    print(validate_model(ySG, pred))
''' 

'\nfor m in models:\n    pred = m.predict(xSG)\n    print(validate_model(ySG, pred))\n'

##Pruning

Iteratively prune the each base badnet models 3 times with <code>prune_low_magenitue</code> in Keras. The clean Validation set is used for fit. Pruned nets are saved in list <code>pruned_models</code>

In [13]:

pruned_models = []
for i, base_model in enumerate(models):
    for _ in range(3):
        model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)
    
        log_dir = tempfile.mkdtemp()
        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
        ]
        model_for_pruning.compile(
          loss=tf.keras.losses.categorical_crossentropy,
          optimizer='adam',
          metrics=['accuracy']
        )
        model_for_pruning.fit(xValid, yValid,
                              callbacks=callbacks,
                              epochs=3
                              )

    pruned_models.append(model_for_pruning)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/3
361/361 [==============================] - 9s 17ms/step - loss: 0.4738 - accuracy: 0.9015
Epoch 2/3
361/361 [==============================] - 6s 16ms/step - loss: 0.1117 - accuracy: 0.9747
Epoch 3/3
361/361 [==============================] - 6s 16ms/step - loss: 0.0563 - accuracy: 0.9849
Epoch 1/3
361/361 [==============================] - 9s 17ms/step - loss: 0.1135 - accuracy: 0.9685
Epoch 2/3
361/361 [==============================] - 6s 16ms/step - loss: 0.0616 - accuracy: 0.9818
Epoch 3/3
361/361 [==============================] - 6s 16ms/step - loss: 0.0862 - accuracy: 0.9773
Epoch 1/3
361/361 [==============================] - 9s 17ms/step - loss: 0.0789 - accuracy: 0.9789
Epoch 2/3
361/361 [==============================] - 6s 17ms/step - loss: 0.0460 - accuracy: 0.9876
Epoch 3/3
361/361 [==============================] - 6s 16ms/step - loss: 0.0366 - accuracy: 0.9886
Epoch 1/3
361/361 [==============================] - 9s 17ms/step - loss: 0.4247 - accuracy: 0.9020


In [14]:
for i, model in enumerate(pruned_models):
    print("model ", i, ":")
    model.summary()

model  0 :
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv_1 (Pru (None, 52, 44, 20)   1942        input[0][0]                      
__________________________________________________________________________________________________
prune_low_magnitude_pool_1 (Pru (None, 26, 22, 20)   1           prune_low_magnitude_conv_1[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_conv_2 (Pru (None, 24, 20, 40)   14442       prune_low_magnitude_pool_1[0][0] 
_________________________________________________________________________________

##Fine-tuning
Fine-tuning the pruned models with <code>1e-5</code> learning rate in 10 epochs. The clean validation set is used in the training process. Repaired nets are saved in list <code>repaired_models</code>

In [15]:
repaired_models = []

for i, m in enumerate(pruned_models):

    model = copy.copy(m)

    print("Fine-tune model", model_names[i], ": ")
    model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy']
              )
    model.fit(xValid, yValid,
          epochs=10
          )
    repaired_models.append(model)

Fine-tune model anonymous_1_bd_net : 
Epoch 1/10
361/361 [==============================] - 7s 13ms/step - loss: 0.0849 - accuracy: 0.9767
Epoch 2/10
361/361 [==============================] - 5s 13ms/step - loss: 0.0448 - accuracy: 0.9874
Epoch 3/10
361/361 [==============================] - 5s 13ms/step - loss: 0.0247 - accuracy: 0.9918
Epoch 4/10
361/361 [==============================] - 5s 13ms/step - loss: 0.0258 - accuracy: 0.9928
Epoch 5/10
361/361 [==============================] - 5s 13ms/step - loss: 0.0147 - accuracy: 0.9962
Epoch 6/10
361/361 [==============================] - 5s 13ms/step - loss: 0.0111 - accuracy: 0.9976
Epoch 7/10
361/361 [==============================] - 5s 13ms/step - loss: 0.0084 - accuracy: 0.9981
Epoch 8/10
361/361 [==============================] - 5s 13ms/step - loss: 0.0058 - accuracy: 0.9989
Epoch 9/10
361/361 [==============================] - 5s 13ms/step - loss: 0.0047 - accuracy: 0.9995
Epoch 10/10
361/361 [==============================] 

##Validation

In [16]:
##Intermediate quick evalution using built-in Keras functions and the poisoned set

for i, m in enumerate(repaired_models):
    print("Repaired", model_names[i], "accuracy on clean test set: ")
    m.evaluate(xTest, yTest) 
    print("Repaired", model_names[i], "accuracy on sunglass poisoned set: ")
    m.evaluate(xSG, ySG)
    
    print("Repaired", model_names[i], "accuracy on anonumous poisoned set: ")
    m.evaluate(xAnon, yAnon) 

Repaired anonymous_1_bd_net accuracy on clean test set: 
401/401 [==============================] - 2s 4ms/step - loss: 0.6486 - accuracy: 0.9212
Repaired anonymous_1_bd_net accuracy on sunglass poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 62.0128 - accuracy: 0.0000e+00
Repaired anonymous_1_bd_net accuracy on anonumous poisoned set: 
321/321 [==============================] - 1s 4ms/step - loss: 22.8137 - accuracy: 0.0083
Repaired multi_trigger_multi_target_bd_net accuracy on clean test set: 
401/401 [==============================] - 2s 4ms/step - loss: 0.5748 - accuracy: 0.9256
Repaired multi_trigger_multi_target_bd_net accuracy on sunglass poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 70.8417 - accuracy: 0.0000e+00
Repaired multi_trigger_multi_target_bd_net accuracy on anonumous poisoned set: 
321/321 [==============================] - 1s 4ms/step - loss: 43.1028 - accuracy: 3.8971e-04
Repaired sunglasses_bd_net accura

##Final output
Compare preprocessed results from the bad net and results from repaired nets to output results. The differnt predictions between bad nets and repaired nets are classifed as N+1 class.

In [17]:
del xValid
del yValid

In [18]:
def poisoned_detection(afterPreds, prePreds):
    out = []
    for i in range(len(afterPreds)):
        newDimension = list(afterPreds[i].shape)
        newDimension[1] += 1
        newArray = np.zeros(newDimension)
        for j in range(afterPreds[i].shape[0]):
            a = np.argmax(afterPreds[i][j]) 
            b = np.argmax(prePreds[i][j])
            
            if a == b:
                newArray[j][a] = 1
            else:
                newArray[j][-1] = 1
        out.append(newArray)

    return out

In [27]:
preds_repaired = [] ##test result of the badnets after repair


for i, m in enumerate(repaired_models):
    if "sunglasses" in model_names[i].lower(): ##only save output from sunglasses backdoored model to save RAM usage 
        pred = m.predict(xSGmixed)
        preds_repaired.append(pred)


In [28]:
final_results = poisoned_detection(preds_repaired, preds_preprocessed)
for r in final_results:
    print(model_names[i], "repaired sunglass_bd_net poisoned detection accuracy", validate_model(ySGmixed, r))
    

anonymous_2_bd_net repaired sunglass_bd_net poisoned detection accuracy 0.4618472330475448
